In [ ]:
import requests
from datetime import datetime, timedelta
from pytz import timezone
import time
import json
import sys

In [ ]:

# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('../server')

from predict_data2 import get_pred_data
from random_forest import predictor

rfp = predictor()

if True: #https://stackoverflow.com/questions/5980042/how-to-implement-the-verbose-or-v-option-into-a-script
    def verboseprint(*args):
        # Print each argument separately so caller doesn't need to
        # stuff everything to be printed into a single string
        print(''.join(map(str, args)))
else:
    warnings.filterwarnings("ignore") #we want to surpress warings
    verboseprint = lambda *a: None

In [ ]:
def toLocal(dt):
    ''' Setzt automatich in die Zeitzone in unsere und convertiert die Zeit'''
    return dt.astimezone(timezone('Europe/Berlin'))
def toUtc(dt):
    ''' Setzt automatich in die Zeitzone'''
    return dt.astimezone(timezone('UTC'))
def setToUTCTime(dt, diff):
    ''' Setzt die Zeitzone des Datetime objects auf utc, diff ist die differenz zu utc'''
    #also um bsp Berlin ist die dif 1 weil wir in GMT+1 leben und somit um auf ut zu kommen müssen -1 h machen
    return dt.replace(tzinfo=timezone('UTC')) - timedelta(hours=diff)
def fromUnix(unix):
    ''' Remove millisecons (/1000) add one hour (+3600) and set timezone'''
    return datetime.utcfromtimestamp(float(unix)/1000 + 3600).replace(tzinfo=timezone('Europe/Berlin'))

def getStation(station):
    ''' Gibt die ersten 24 stastionen zurück'''
    url = "https://marudor.de/api/hafas/v1/station/" + station + "?profile=db"
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()
    else:
        return False

def getCon(start, destination, time):
    #marudor will unix zeit in UTC zeit aber .timestamp macht des automatich wenn die Zeitzone gesetzt ist
    #json={"start":getStation(start)[0]['id'],"destination":getStation(destination)[0]['id'],"time":time.timestamp()*1000,"maxChanges":"-1","transferTime":"0","hafasProfile":"db"}
    json = {"start":"008000141","destination":"008000207","time":1572961200000,"maxChanges":"-1","transferTime":"0","hafasProfile":"db"}
    print(json)
    r = requests.post('https://marudor.de/api/hafas/v1/route?profile=db', json=json)
    print(r.status_code)
    return r.text

def cleanCon(connection):
    for i in range(len(connection)):
        connection[i]['segments'].append({ 'segmentTypes': connection[i]['segmentTypes']})
        print(connection[i]['segments'])
        connection[i] = connection[i]['segments']
        for n in range(len(connection[i])):
            if 'wings' in connection[i][n]:
                #print("deleting wings")
                del connection[i][n]['wings']
            if 'messages' in connection[i][n]:
                #print("deleting messages")
                del connection[i][n]['messages']
            if 'jid' in connection[i][n]:
                #print("deleting jid")
                del connection[i][n]['jid']
            if 'type' in connection[i][n]:
                #print("deleting type")
                del connection[i][n]['type']
            if 'stops' in connection[i][n]:
                #print("deleting stops")
                del connection[i][n]['stops']
            if 'finalDestination' in connection[i][n]:
                #print("deleting finalDestination")
                del connection[i][n]['finalDestination']
    return connection

In [ ]:
#nur nen modell, das Zeigt auch wenn wir in unserer Zeitzone sind mach des kein unterschied
connectionb = getCon("Tübingen Hbf", "Köln Hbf", datetime(2019, 11, 5, 14, 41, 43, 558646, tzinfo=timezone('UTC')))

In [ ]:
def analysis(connection):
    #verboseprint('Mache Analyse...')
    #score = 0
    total_score = 0
    #change between scheduled and predicted time
    time = 'scheduledTime' # 'time'
    
    #there will always be one segment than connections
    for i in range(len(connection)-1):
        #I think we can delet this?
        if connection[i] == '':
            verboseprint("No data for connection = "+ str(i))
            continue

        data1 = get_pred_data(connection[i]['segmentDestination']['title'], connection[i]['train']['name'], fromUnix(connection[i]['arrival'][time]))
        data2 = get_pred_data(connection[i+1]['segmentStart']['title'], connection[i+1]['train']['name'], fromUnix(connection[i+1]['departure'][time]))
        transtime = ((fromUnix(connection[i+1]['departure'][time]) - fromUnix(connection[i]['arrival'][time])).seconds//60)%60 #we just want minutes
        print(transtime)
        print(data1)
        print(data2)
        connection[i]['con_score'], connection[i]['adelay5'], connection[i+1]['ddelay5'] = rfp.predict_con(data1, data2, transtime)

        verboseprint("Score for connection[", connection[i]['train']['name'], ' to ', connection[i+1]['train']['name'], ' in ', connection[i]['segmentDestination']['title'],"] = " ,connection[i]['con_score'])
        if (total_score == 0):
            total_score = connection[i]['con_score']
        else:
            total_score *= connection[i]['con_score']

    #score = total_score / (len(connection) - 1)
    verboseprint('\n RETURNING SCORE:' + str(total_score))
    
    connection.append({'total_score': int(total_score * 100)})
    return connection

In [ ]:
connection = json.loads(connectionb)["routes"]
connection = cleanCon(connection)
#connection[0]

In [ ]:
print(connection[0])
analysis(connection[0])

In [ ]:
test = connection[0]['segments']
test

In [ ]:
(connection[0]['segments'].append({ 'segmentTypes': connection[0]['segmentTypes']}))

In [ ]:
connection[0]